In [ ]:
counter = 0
import webrtcvad
import wave
import pyaudio
import webrtcvad
import pyaudio
import wave
import os, sys
import librosa
import numpy as np
import matplotlib.pyplot as plt
import librosa.display
import webbrowser
import time
import speech_recognition as sr
import time
from gtts import gTTS
import random

In [ ]:
def record_audio():
    global counter 
    counter+=1
    vad = webrtcvad.Vad()
    vad.set_mode(2)
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 32000
    CHUNK = 960
    RECORD_SECONDS = 4
    WAVE_OUTPUT_FILENAME = "order/file"+str(counter)+".wav"
    audio = pyaudio.PyAudio()
    # start Recording
    stream = audio.open(format=FORMAT, channels=CHANNELS,
                    rate=RATE, input=True,
                    frames_per_buffer=CHUNK)
    while True:

        frames = []
        frameCount = 0

        while frameCount:
            data = stream.read(CHUNK)

            if vad.is_speech(data, RATE):
                frameCount+=1;
            else:
                frameCount = 0;
            # print frameCount

        print ("recording...")
        for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
            data = stream.read(CHUNK)
            frames.append(data)
        print("finished recording")

        waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
        waveFile.setnchannels(CHANNELS)
        waveFile.setsampwidth(audio.get_sample_size(FORMAT))
        waveFile.setframerate(RATE)
        waveFile.writeframes(b''.join(frames))
        waveFile.close()

        os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

        # change this as you see fit
        audio_path = "order/file"+str(counter)+".wav"

        y, sr = librosa.load(audio_path)
        break;
    # stop Recording
    stream.stop_stream()
    stream.close()
    audio.terminate()
    return audio_path

In [ ]:
import speech_recognition as sr
print(sr.__version__)

In [ ]:
import pyaudio as p
print(p.__version__)

In [ ]:
r = sr.Recognizer()

In [ ]:
import datetime


def record(ask = False):
    if ask:
        speak(ask)
    path = record_audio()
    print(path)
    voice = sr.AudioFile(path)
    with voice as source:
        audio = r.record(source)
        voice_data = ''
        try:
            voice_data = r.recognize_google(audio)  # convert audio to text
        except sr.UnknownValueError: # error: recognizer does not understand
            speak('I did not get that')
        except sr.RequestError:
            speak('Sorry, the service is down') # error: recognizer is not connected
        print(f"{voice_data.lower()}") # print what user said
        return voice_data.lower()
    
def responde(voice_data):
    if "your name" in voice_data:
        speak('my name is mooohsseeeennaaaa')
    if "bye" in voice_data:
        speak('see you')
    if "what time" in voice_data:
        speak(datetime.datetime.now().strftime("%b %d %Y %H:%M:%S"))
    if "search" in voice_data:
        search = record('What do you want to search for?')
        url = "https://google.com/search?q="+search
        webbrowser.get().open(url)
        speak('Here is what I found for ' + search + " on google")
    if "location" in voice_data:
        location = record('What is the location ?')
        url = "https://google.nl/maps/place/"+location+"/&amp;"
        webbrowser.get().open(url)
        speak('Here is the location of ' + location + " on google")
    if "exit" in voice_data:
        exit()
        
def speak(audio_string):
    tts = gTTS(text=audio_string, lang='en')
    r = random.randint(1,20000000)
    audio_file = 'audio' + str(r) + '.mp3'
    tts.save(audio_file)
    playsound(audio_file)
    print(audio_string) 
    os.remove(audio_file) 
        
time.sleep(1)    
speak('How can I help you?')
while 1:
    voice_data = record()
    responde(voice_data)
